In [ ]:
# Import required libraries
import pandas as pd
import numpy as np
import os
import json
from pathlib import Path
import warnings
warnings.filterwarnings('ignore')

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)

In [ ]:
# Define paths - UPDATE THESE TO YOUR ABCD DATA LOCATION
ABCD_DATA_PATH = "/path/to/abcd/data"  # UPDATE THIS PATH
OUTPUT_DIR = "/Users/chloehampson/Desktop/hippo-amyg-depression/derivatives"

os.makedirs(OUTPUT_DIR, exist_ok=True)
print(f"Output directory: {OUTPUT_DIR}")

## 1. Define All Variables (from ABCD Data Dictionary)

In [ ]:
# Subject identifier
SUBJECT_ID = 'src_subject_id'
SESSION_ID = 'eventname'

# =====================================================
# COVARIATE VARIABLES (for residualization)
# =====================================================
COVARIATES = {
    'age': 'ab_p_demo_age',                       # Youth's age at data collection
    'sex': 'ab_g_stc__cohort_sex',                # Participant's sex
    'ethnicity': 'ab_g_stc__cohort_ethnrace__meim',  # Ethno-racial identity (15 levels)
    'race': 'ab_g_stc__cohort_race__nih',         # Race (NIH 7 levels)
    'site': 'ab_g_dyn__design_site',              # Assessment site
    'family_id': 'ab_g_stc__design_id__fam',      # Family ID (for nesting)
    # 'head_motion': 'mr_y_qc__mot',              # TBD based on task
}

print(f"Covariates: {len(COVARIATES)}")
for k, v in COVARIATES.items():
    print(f"  {k}: {v}")

In [ ]:
# =====================================================
# SOCIOENVIRONMENTAL VARIABLES (PLSC Block 1 / X)
# =====================================================

# Family Environment (fc_p_fes)
FAMILY_ENV_VARS = [
    'fc_p_fes__cohes_mean',      # Family Environment Scale—Cohesion: Mean
    'fc_p_fes__confl_mean',      # Family Environment Scale—Conflict: Mean
    'fc_p_fes__expr_mean',       # Family Environment Scale—Expression: Mean
    'fc_p_fes__intelcult_mean',  # Family Environment Scale—Intellectual/cultural: Mean
    'fc_p_fes__org_mean',        # Family Environment Scale—Organization: Mean
    'fc_p_fes__rec_mean',        # Family Environment Scale—Activity/recreational: Mean
]

# Cultural & Social Environment
CULTURAL_SOCIAL_VARS = [
    'fc_y_meim_mean',                          # Multigroup Ethnic Identity Measure [Youth]: Mean
    'fc_p_meim_mean',                          # Multigroup Ethnic Identity Measure [Parent]: Mean
    'le_l_nbhsoc__addr1__factor3_score',       # Neighborhood ethnic/immigrant concentration
    'fc_y_srpf__env_mean',                     # School Risk & Protective Factors—School environment: Mean
]

# Socioeconomic Status & Demographics
SES_VARS = [
    'le_l_nbhsoc__addr1__factor1_score',       # Neighborhood disadvantage score
    'le_l_nbhsoc__addr1__aff_score',           # Neighborhood affluence score
    'le_l_coi__addr1__coi__total__metro_score',# Child Opportunity Index (metro-normed, 1-100)
    'ab_p_demo__income__hhold_001',            # Household income category
    'ab_g_dyn__cohort_edu__cgs',               # Highest education across caregivers
]

# Combine all socioenvironmental
SOCIOENV_VARS = FAMILY_ENV_VARS + CULTURAL_SOCIAL_VARS + SES_VARS

print(f"\nSOCIOENVIRONMENTAL VARIABLES: {len(SOCIOENV_VARS)}")
print(f"\n  Family Environment ({len(FAMILY_ENV_VARS)}):")
for v in FAMILY_ENV_VARS: print(f"    - {v}")
print(f"\n  Cultural/Social ({len(CULTURAL_SOCIAL_VARS)}):")
for v in CULTURAL_SOCIAL_VARS: print(f"    - {v}")
print(f"\n  SES ({len(SES_VARS)}):")
for v in SES_VARS: print(f"    - {v}")

In [ ]:
# =====================================================
# DEPRESSION VARIABLES (PLSC Block 2 / Y)
# =====================================================

DEPRESSION_VARS = [
    # Summary scores
    'mh_p_cbcl__dsm__dep_sum',      # CBCL [Parent] DSM-5 Depressive problems: Sum
    'mh_y_ysr__dsm__dep_sum',       # YSR [Youth] DSM-5 Depressive problems: Sum
    
    # Core symptoms - Parent report (KSADS)
    'mh_p_ksads__dep__mood__pres_sx',   # Depressed mood - Present [Parent]
    'mh_p_ksads__dep__anhed__pres_sx',  # Anhedonia - Present [Parent]
    'mh_p_ksads__dep__fatig__pres_sx',  # Fatigue - Present [Parent]
    
    # Core symptoms - Youth report (KSADS)
    'mh_y_ksads__dep__mood__pres_sx',   # Depressed mood - Present [Youth]
    'mh_y_ksads__dep__anhed__pres_sx',  # Anhedonia - Present [Youth]
    'mh_y_ksads__dep__fatig__pres_sx',  # Fatigue - Present [Youth]
]

print(f"DEPRESSION VARIABLES: {len(DEPRESSION_VARS)}")
for v in DEPRESSION_VARS:
    print(f"  - {v}")

In [ ]:
# =====================================================
# BRAIN VARIABLES - 4 ROIs (Stage 2 - Analyzed Separately)
# =====================================================
# Emotional N-back task beta weights

# Hippocampus: 2-back vs 0-back contrast (working memory load)
HIPPOCAMPUS_VARS = {
    'hippo_lh': 'mr_y_tfmri__nback__2bv0b__aseg__hc__lh_beta',  # Left hippocampus
    'hippo_rh': 'mr_y_tfmri__nback__2bv0b__aseg__hc__rh_beta',  # Right hippocampus
}

# Amygdala: Emotion vs Neutral face contrast (emotional processing)
AMYGDALA_VARS = {
    'amyg_lh': 'mr_y_tfmri__nback__emovntf__aseg__ag__lh_beta',  # Left amygdala
    'amyg_rh': 'mr_y_tfmri__nback__emovntf__aseg__ag__rh_beta',  # Right amygdala
}

# Combined dictionary for all brain ROIs
BRAIN_ROIS = {**HIPPOCAMPUS_VARS, **AMYGDALA_VARS}

# List of variable names for data loading
BRAIN_VARS = list(BRAIN_ROIS.values())

# Labels for plotting/output
BRAIN_ROI_LABELS = {
    'mr_y_tfmri__nback__2bv0b__aseg__hc__lh_beta': 'Left Hippocampus (2b>0b)',
    'mr_y_tfmri__nback__2bv0b__aseg__hc__rh_beta': 'Right Hippocampus (2b>0b)',
    'mr_y_tfmri__nback__emovntf__aseg__ag__lh_beta': 'Left Amygdala (emo>neut)',
    'mr_y_tfmri__nback__emovntf__aseg__ag__rh_beta': 'Right Amygdala (emo>neut)',
}

print(f"BRAIN ROIs: {len(BRAIN_VARS)} (analyzed separately in Stage 2)")
print("\n  Hippocampus (2-back vs 0-back - working memory):")
for k, v in HIPPOCAMPUS_VARS.items():
    print(f"    - {k}: {v}")
print("\n  Amygdala (emotion vs neutral face - emotional processing):")
for k, v in AMYGDALA_VARS.items():
    print(f"    - {k}: {v}")

## 2. Load ABCD Data Tables

In [ ]:
def load_abcd_table(table_name, data_path=ABCD_DATA_PATH):
    """Load an ABCD data table. Tries .csv, .txt, .tsv extensions."""
    for ext in ['.csv', '.txt', '.tsv']:
        filepath = os.path.join(data_path, f"{table_name}{ext}")
        if os.path.exists(filepath):
            sep = '\t' if ext == '.tsv' else ','
            return pd.read_csv(filepath, sep=sep, low_memory=False)
    print(f"Warning: Could not find table {table_name}")
    return None

print("Table loading function defined.")

In [ ]:
# Load data tables - UPDATE table names based on your ABCD data release
# These are the table names from your data dictionary

tables_to_load = {
    # Demographics
    'ab_p_demo': 'ab_p_demo',       # Parent demographics (age, income)
    'ab_g_stc': 'ab_g_stc',         # Static cohort info (sex, race, family ID)
    'ab_g_dyn': 'ab_g_dyn',         # Dynamic design info (site, education)
    
    # Family Environment Scale
    'fc_p_fes': 'fc_p_fes',         # Family Environment Scale [Parent]
    
    # Cultural & Social
    'fc_y_meim': 'fc_y_meim',       # MEIM [Youth]
    'fc_p_meim': 'fc_p_meim',       # MEIM [Parent]
    'le_l_nbhsoc': 'le_l_nbhsoc',   # Neighborhood social
    'fc_y_srpf': 'fc_y_srpf',       # School Risk & Protective Factors
    
    # SES
    'le_l_coi': 'le_l_coi',         # Child Opportunity Index
    
    # Depression
    'mh_p_cbcl': 'mh_p_cbcl',       # CBCL [Parent]
    'mh_y_ysr': 'mh_y_ysr',         # YSR [Youth]
    'mh_p_ksads__dep': 'mh_p_ksads__dep',  # KSADS Depression [Parent]
    'mh_y_ksads__dep': 'mh_y_ksads__dep',  # KSADS Depression [Youth]
    
    # MRI
    'mr_y_qc': 'mr_y_qc',           # MRI Quality Control
    'mr_y_tfmri__nback__2bv0b__aseg': 'mr_y_tfmri__nback__2bv0b__aseg',  # N-back 2b>0b (hippocampus)
    'mr_y_tfmri__nback__emovntf__aseg': 'mr_y_tfmri__nback__emovntf__aseg',  # N-back emo>neut (amygdala)
}

# Load all tables
loaded_tables = {}
for name, table in tables_to_load.items():
    loaded_tables[name] = load_abcd_table(table)

# Check what loaded
print("\nTable loading status:")
for name, df in loaded_tables.items():
    if df is not None:
        print(f"  ✓ {name}: {len(df)} rows, {len(df.columns)} columns")
    else:
        print(f"  ✗ {name}: NOT FOUND")

## 3. Merge Data Tables

In [ ]:
def merge_abcd_tables(tables_dict, on_cols=['src_subject_id', 'eventname'], how='inner'):
    """Merge multiple ABCD tables on subject ID and eventname."""
    merged_df = None
    
    for name, df in tables_dict.items():
        if df is None:
            continue
            
        available_cols = [col for col in on_cols if col in df.columns]
        if len(available_cols) == 0:
            print(f"Skipping {name} - No merge columns found")
            continue
        
        if merged_df is None:
            merged_df = df.copy()
            print(f"Initialized with {name}: {len(merged_df)} rows")
        else:
            merged_df = merged_df.merge(df, on=available_cols, how=how, suffixes=('', f'_{name}'))
            print(f"After merging {name}: {len(merged_df)} rows")
    
    return merged_df

# Filter to tables that loaded successfully
valid_tables = {k: v for k, v in loaded_tables.items() if v is not None}
print(f"\nTables available for merging: {len(valid_tables)}")

In [ ]:
# Merge tables
# Uncomment when data is loaded:
# merged_df = merge_abcd_tables(valid_tables)
# print(f"\nFinal merged dataset: {len(merged_df)} rows, {len(merged_df.columns)} columns")

print("NOTE: Uncomment the merge function once data is loaded")

## 4. Select Timepoint & Apply QC Filters

In [ ]:
# Select timepoint - many variables available at ses-02A (2-year follow-up)
TIMEPOINT = 'ses-02A'  # UPDATE AS NEEDED

# Alternative timepoints:
# 'ses-00A' = Baseline
# 'ses-02A' = 2-year follow-up
# 'ses-04A' = 4-year follow-up

print(f"Selected timepoint: {TIMEPOINT}")

In [ ]:
def apply_qc_filters(df, motion_threshold=0.5, motion_var=None, qc_var=None):
    """Apply quality control filters for neuroimaging data."""
    n_initial = len(df)
    
    if qc_var and qc_var in df.columns:
        df = df[df[qc_var] == 1]
        print(f"After QC inclusion: {len(df)} ({n_initial - len(df)} removed)")
    
    if motion_var and motion_var in df.columns:
        n_before = len(df)
        df = df[df[motion_var] <= motion_threshold]
        print(f"After motion filter (≤{motion_threshold}): {len(df)} ({n_before - len(df)} removed)")
    
    return df

# Uncomment when data is loaded:
# # Filter to timepoint
# analysis_df = merged_df[merged_df['eventname'] == TIMEPOINT].copy()
# print(f"Subjects at {TIMEPOINT}: {len(analysis_df)}")
# 
# # Apply QC filters
# analysis_df = apply_qc_filters(analysis_df)

print("QC filter function defined.")

## 5. Create PLSC Dataframes

In [ ]:
def create_plsc_dataframes(df, socioenv_vars, depression_vars, brain_vars, 
                           covariate_vars, subject_id='src_subject_id'):
    """
    Create FOUR separate dataframes for the two-stage PLSC analysis.
    
    Stage 1 PLSC:
      1. Socioenvironmental (Block 1 / X)
      2. Depression (Block 2 / Y)
    
    Stage 2 (Brain - analyzed separately):
      3. Brain ROIs (4 variables)
    
    Plus:
      4. Covariates (for residualization)
    """
    
    # Filter to existing variables
    existing_socioenv = [v for v in socioenv_vars if v in df.columns]
    existing_depression = [v for v in depression_vars if v in df.columns]
    existing_brain = [v for v in brain_vars if v in df.columns]
    existing_covars = {k: v for k, v in covariate_vars.items() if v in df.columns}
    
    print(f"Variables found:")
    print(f"  Socioenvironmental: {len(existing_socioenv)}/{len(socioenv_vars)}")
    print(f"  Depression: {len(existing_depression)}/{len(depression_vars)}")
    print(f"  Brain ROIs: {len(existing_brain)}/{len(brain_vars)}")
    print(f"  Covariates: {len(existing_covars)}/{len(covariate_vars)}")
    
    # Report missing variables
    missing_socioenv = [v for v in socioenv_vars if v not in df.columns]
    missing_depression = [v for v in depression_vars if v not in df.columns]
    missing_brain = [v for v in brain_vars if v not in df.columns]
    
    if missing_socioenv:
        print(f"\n  Missing socioenvironmental: {missing_socioenv}")
    if missing_depression:
        print(f"  Missing depression: {missing_depression}")
    if missing_brain:
        print(f"  Missing brain: {missing_brain}")
    
    # Create covariate dataframe
    covar_cols = [subject_id] + list(existing_covars.values())
    covar_df = df[covar_cols].copy().set_index(subject_id)
    
    # Create socioenvironmental dataframe (PLSC Block 1 / X)
    socioenv_df = df[[subject_id] + existing_socioenv].copy().set_index(subject_id)
    
    # Create depression dataframe (PLSC Block 2 / Y)
    depression_df = df[[subject_id] + existing_depression].copy().set_index(subject_id)
    
    # Create brain dataframe (Stage 2 - 4 ROIs)
    brain_df = df[[subject_id] + existing_brain].copy().set_index(subject_id)
    
    return {
        'covariates': covar_df,
        'socioenv': socioenv_df,
        'depression': depression_df,
        'brain': brain_df,
        'variable_lists': {
            'socioenv': existing_socioenv,
            'depression': existing_depression,
            'brain': existing_brain,
            'covariates': existing_covars,
            'brain_labels': BRAIN_ROI_LABELS
        }
    }

print("PLSC dataframe creation function defined.")

In [ ]:
# Create PLSC dataframes
# Uncomment when analysis_df is available:

# plsc_data = create_plsc_dataframes(
#     analysis_df,
#     socioenv_vars=SOCIOENV_VARS,
#     depression_vars=DEPRESSION_VARS,
#     brain_vars=BRAIN_VARS,
#     covariate_vars=COVARIATES
# )
# 
# print("\nDataframe shapes:")
# print(f"  Covariates: {plsc_data['covariates'].shape}")
# print(f"  Socioenvironmental (PLSC Block 1): {plsc_data['socioenv'].shape}")
# print(f"  Depression (PLSC Block 2): {plsc_data['depression'].shape}")
# print(f"  Brain - 4 ROIs (Stage 2): {plsc_data['brain'].shape}")

## 6. Handle Missing Data

In [ ]:
def assess_and_handle_missing(plsc_data, threshold=0.2):
    """
    Assess missing data and apply listwise deletion.
    Reports variables with high missingness.
    """
    print("Missing data assessment:")
    
    for name, df in plsc_data.items():
        if name == 'variable_lists':
            continue
            
        print(f"\n{name.upper()}:")
        for col in df.columns:
            pct_missing = df[col].isna().mean() * 100
            if pct_missing > 0:
                flag = " ⚠️ HIGH" if pct_missing > threshold * 100 else ""
                print(f"  {col}: {pct_missing:.1f}% missing{flag}")
    
    # Get common subjects across all dataframes
    common_idx = plsc_data['covariates'].index
    for name, df in plsc_data.items():
        if name != 'variable_lists':
            common_idx = common_idx.intersection(df.dropna().index)
    
    print(f"\nSubjects with complete data: {len(common_idx)}")
    
    # Filter all dataframes to common subjects
    for name in ['covariates', 'socioenv', 'depression', 'brain']:
        plsc_data[name] = plsc_data[name].loc[common_idx]
    
    return plsc_data

# Uncomment when plsc_data is available:
# plsc_data = assess_and_handle_missing(plsc_data)

print("Missing data function defined.")

## 7. Save Dataframes for R Analysis

In [ ]:
def save_plsc_dataframes(plsc_data, output_dir):
    """
    Save PLSC dataframes to CSV files for R analysis.
    """
    os.makedirs(output_dir, exist_ok=True)
    
    # Save covariates
    plsc_data['covariates'].to_csv(os.path.join(output_dir, 'covariate.csv'))
    print(f"Saved: covariate.csv {plsc_data['covariates'].shape}")
    
    # Save socioenvironmental (PLSC Block 1)
    plsc_data['socioenv'].to_csv(os.path.join(output_dir, 'clean-socioenv.csv'))
    print(f"Saved: clean-socioenv.csv {plsc_data['socioenv'].shape}")
    
    # Save depression (PLSC Block 2)
    plsc_data['depression'].to_csv(os.path.join(output_dir, 'clean-depression.csv'))
    print(f"Saved: clean-depression.csv {plsc_data['depression'].shape}")
    
    # Save brain data (Stage 2 - 4 ROIs)
    plsc_data['brain'].to_csv(os.path.join(output_dir, 'clean-brain-hippo-amyg.csv'))
    print(f"Saved: clean-brain-hippo-amyg.csv {plsc_data['brain'].shape}")
    
    # Save variable lists as JSON
    with open(os.path.join(output_dir, 'variable_lists.json'), 'w') as f:
        json.dump(plsc_data['variable_lists'], f, indent=2)
    print("Saved: variable_lists.json")
    
    print(f"\nAll files saved to: {output_dir}")

# Uncomment when plsc_data is available:
# save_plsc_dataframes(plsc_data, OUTPUT_DIR)

print("Save function defined.")

## Summary

### Files to be Created:
| File | Contents | Purpose |
|------|----------|--------|
| `covariate.csv` | Age, sex, race, site, family ID | Residualization |
| `clean-socioenv.csv` | 15 socioenvironmental vars | PLSC Block 1 (X) |
| `clean-depression.csv` | 8 depression vars | PLSC Block 2 (Y) |
| `clean-brain-hippo-amyg.csv` | 4 brain ROIs | Stage 2 (separate) |

### Analysis Flow:
```
STAGE 1: PLSC
   Socioenvironmental (15 vars) ←→ Depression (8 vars)
   → Identifies latent dimensions
   → Extract LV scores

STAGE 2: Brain (4 separate analyses)
   LV scores ←→ Left Hippocampus (2b>0b)
   LV scores ←→ Right Hippocampus (2b>0b)
   LV scores ←→ Left Amygdala (emo>neut)
   LV scores ←→ Right Amygdala (emo>neut)
```

In [ ]:
print("="*60)
print("DATA PREPARATION SCRIPT READY")
print("="*60)
print(f"\n1. Update ABCD_DATA_PATH at the top of this notebook")
print(f"2. Run cells to load and merge data")
print(f"3. Output will be saved to: {OUTPUT_DIR}")
print(f"4. Then run hippo_amyg_depression_plsc.Rmd in R")